In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary

%matplotlib inline

Using TensorFlow backend.


In [2]:
from s2sutils import *

In [3]:
# load models
dictionary = Dictionary.load('chartovec_big.dict')
chartovec_encoder = SentenceToCharVecEncoder(dictionary)

model = keras.models.model_from_json(open('s2s_big.json', 'rb').read())
model.load_weights('s2s_big.h5')

encoder_model = keras.models.model_from_json(open('s2s_encoder_big.json', 'rb').read())
encoder_model.load_weights('s2s_encoder_big.h5')

decoder_model = keras.models.model_from_json(open('s2s_decoder_big.json', 'rb').read())
decoder_model.load_weights('s2s_decoder_big.h5')

In [4]:
numchars = len(chartovec_encoder.dictionary)
latent_dim = numchars + 20

print numchars
print latent_dim

93
113


In [5]:
def decode_sequence(input_sent, dictionary, maxlen=20, num_chars=93):
    # Encode the input as state vectors.
    input_seq = np.array([chartovec_encoder.encode_sentence(input_sent, endsig=True).toarray()])
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_chars))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, dictionary.token2id['\n']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = dictionary[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > maxlen):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_chars))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [6]:
decode_sequence('Happy Holiday!', dictionary)

u'In the propose of th\n'

In [7]:
decode_sequence('Merry Christmas!', dictionary)

u'"What are you have b\n'

In [8]:
decode_sequence('I love natural language processing and sequence to sequence model.', dictionary)

u'later and the soldie\n'

In [9]:
decode_sequence('Python or R?', dictionary)

u'In the propose of th\n'